# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [1]:
# Your code here; import the necessary packages
import sqlite3

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [6]:
# Your code here

try:
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    print("Connected to the database successfully!")

    # Execute the query using a subquery
    cursor.execute("""
        SELECT
            customerNumber,
            contactLastName,
            contactFirstName
        FROM customers
        WHERE customerNumber IN (
            SELECT customerNumber
            FROM orders
            WHERE orderDate = '2003-01-31'
        )
    """)
    
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    connection.close()
    print("Connection closed.")

except sqlite3.Error as e:
    print("Error connecting to the database:", e)


Connected to the database successfully!
(141, 'Freyre', 'Diego ')
Connection closed.


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [7]:
# Your code here
try:
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    print("Connected to the database successfully!")

    cursor.execute("""
        SELECT
            p.productCode,
            p.productName,
            SUM(od.quantityOrdered) AS totalItemsSold
        FROM products AS p
        JOIN orderdetails AS od
            ON p.productCode = od.productCode
        JOIN orders AS o
            ON od.orderNumber = o.orderNumber
        WHERE o.orderDate = '2003-01-31'
        GROUP BY p.productCode, p.productName
        ORDER BY totalItemsSold DESC;
    """)
    
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    connection.close()
    print("Connection closed.")

except sqlite3.Error as e:
    print("Error connecting to the database:", e)

Connected to the database successfully!
('S32_3207', "1950's Chicago Surface Lines Streetcar", 49)
('S24_2840', '1958 Chevy Corvette Limited Edition', 44)
('S12_4473', '1957 Chevy Pickup', 41)
('S18_4027', '1970 Triumph Spitfire', 38)
('S24_1444', '1970 Dodge Coronet', 35)
('S32_2509', '1954 Greyhound Scenicruiser', 35)
('S12_3148', '1969 Corvair Monza', 34)
('S50_1392', 'Diamond T620 Semi-Skirted Tanker', 33)
('S50_1514', '1962 City of Detroit Streetcar', 32)
('S18_2319', '1964 Mercedes Tour Bus', 29)
('S24_4048', '1992 Porsche Cayenne Turbo Silver', 26)
('S18_2238', '1998 Chrysler Plymouth Prowler', 24)
('S18_3232', '1992 Ferrari 360 Spider red', 23)
Connection closed.


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [9]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

try:
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    print("Connected to the database successfully!")

    # Execute the query to get the product name and total number of people who ordered each product
    cursor.execute("""
        SELECT p.productName, COUNT(DISTINCT o.customerNumber) AS totalCustomers
        FROM products AS p
        JOIN orderdetails AS od ON p.productCode = od.productCode
        JOIN orders AS o ON od.orderNumber = o.orderNumber
        GROUP BY p.productName
        ORDER BY totalCustomers DESC;
    """)
    
    # Fetch and print the results
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    # Close the database connection
    connection.close()
    print("Connection closed.")

except sqlite3.Error as e:
    print("Error connecting to the database:", e)

Connected to the database successfully!
('1992 Ferrari 360 Spider red', 40)
('Boeing X-32A JSF', 27)
('1972 Alfa Romeo GTA', 27)
('1952 Alpine Renault 1300', 27)
('1934 Ford V8 Coupe', 27)
('HMS Bounty', 26)
('Corsair F4U ( Bird Cage)', 26)
('American Airlines: MD-11S', 26)
('1999 Yamaha Speed Boat', 26)
('1996 Moto Guzzi 1100i', 26)
('1995 Honda Civic', 26)
('1980s Black Hawk Helicopter', 26)
('1969 Harley Davidson Ultimate Chopper', 26)
('1941 Chevrolet Special Deluxe Cabriolet', 26)
('1940 Ford Delivery Sedan', 26)
('1937 Horch 930V Limousine', 26)
('1913 Ford Model T Speedster', 26)
('1900s Vintage Bi-Plane', 26)
('18th Century Vintage Horse Carriage', 26)
('America West Airlines B757-200', 25)
('ATA: B757-300', 25)
('2003 Harley-Davidson Eagle Drag Bike', 25)
('2002 Suzuki XREO', 25)
('1997 BMW R 1100 S', 25)
('1993 Mazda RX-7', 25)
('1962 LanciaA Delta 16V', 25)
('1960 BSA Gold Star DBD34', 25)
('1948 Porsche 356-A Roadster', 25)
('1940s Ford truck', 25)
('1940 Ford Pickup Truck'

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [10]:
# Your code here

try:
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    print("Connected to the database successfully!")

    cursor.execute("""
        SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city AS officeCity, e.officeCode
        FROM employees AS e
        JOIN offices AS o ON e.officeCode = o.officeCode
        JOIN customers AS c ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders AS o2 ON c.customerNumber = o2.customerNumber
        JOIN orderdetails AS od ON o2.orderNumber = od.orderNumber
        WHERE od.productCode IN (
            SELECT DISTINCT od2.productCode
            FROM orderdetails AS od2
            JOIN orders AS o3 ON od2.orderNumber = o3.orderNumber
            GROUP BY od2.productCode
            HAVING COUNT(DISTINCT o3.customerNumber) < 20
        );
    """)
    
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    connection.close()
    print("Connection closed.")

except sqlite3.Error as e:
    print("Error connecting to the database:", e)



Connected to the database successfully!
(1370, 'Gerard', 'Hernandez', 'Paris', 4)
(1501, 'Larry', 'Bott', 'London', 7)
(1337, 'Loui', 'Bondur', 'Paris', 4)
(1166, 'Leslie', 'Thompson', 'San Francisco', 1)
(1286, 'Foon Yue', 'Tseng', 'NYC', 3)
(1612, 'Peter', 'Marsh', 'Sydney', 6)
(1611, 'Andy', 'Fixter', 'Sydney', 6)
(1401, 'Pamela', 'Castillo', 'Paris', 4)
(1621, 'Mami', 'Nishi', 'Tokyo', 5)
(1323, 'George', 'Vanauf', 'NYC', 3)
(1165, 'Leslie', 'Jennings', 'San Francisco', 1)
(1702, 'Martin', 'Gerard', 'Paris', 4)
(1216, 'Steve', 'Patterson', 'Boston', 2)
(1188, 'Julie', 'Firrelli', 'Boston', 2)
(1504, 'Barry', 'Jones', 'London', 7)
Connection closed.


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [11]:
# Your code here

try:
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    print("Connected to the database successfully!")

    cursor.execute("""
        SELECT
            e.employeeNumber,
            e.firstName,
            e.lastName,
            COUNT(c.customerNumber) AS numberOfCustomers
        FROM
            employees AS e
        JOIN
            customers AS c
        ON
            e.employeeNumber = c.salesRepEmployeeNumber
        GROUP BY
            e.employeeNumber,
            e.firstName,
            e.lastName
        HAVING
            AVG(c.creditLimit) > 15000;
    """)

    rows = cursor.fetchall()
    for row in rows:
        print(row)

    connection.close()
    print("Connection closed.")

except sqlite3.Error as e:
    print("Error connecting to the database:", e)


Connected to the database successfully!
(1165, 'Leslie', 'Jennings', 6)
(1166, 'Leslie', 'Thompson', 6)
(1188, 'Julie', 'Firrelli', 6)
(1216, 'Steve', 'Patterson', 6)
(1286, 'Foon Yue', 'Tseng', 7)
(1323, 'George', 'Vanauf', 8)
(1337, 'Loui', 'Bondur', 6)
(1370, 'Gerard', 'Hernandez', 7)
(1401, 'Pamela', 'Castillo', 10)
(1501, 'Larry', 'Bott', 8)
(1504, 'Barry', 'Jones', 9)
(1611, 'Andy', 'Fixter', 5)
(1612, 'Peter', 'Marsh', 5)
(1621, 'Mami', 'Nishi', 5)
(1702, 'Martin', 'Gerard', 6)
Connection closed.


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!